# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tobol,52.6844,62.5794,262.02,99,68,1.97,KZ,1738433540
1,1,olonkinbyen,70.9221,-8.7187,273.74,94,100,18.98,SJ,1738433541
2,2,redencao,-4.2258,-38.7306,305.85,40,78,4.59,BR,1738433543
3,3,udachny,66.4167,112.4000,235.75,100,99,2.14,RU,1738433544
4,4,port-aux-francais,-49.3500,70.2167,278.31,78,89,4.34,TF,1738433545
...,...,...,...,...,...,...,...,...,...,...
605,605,papagaios,-19.4492,-44.7478,297.24,79,100,2.86,BR,1738434301
606,606,el abiodh sidi cheikh,32.8930,0.5484,281.78,51,31,5.58,DZ,1738434304
607,607,kerikeri,-35.2268,173.9474,288.27,95,32,1.97,NZ,1738434305
608,608,taltal,-25.4000,-70.4833,293.91,74,10,1.68,CL,1738434306


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size='Humidity',
    scale=0.75,
    color='City',
    alpha=1, 
    tiles='OSM',
    frame_width=1000,
    frame_height=750
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_cities_list_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) \
    & (city_data_df['Wind Speed'] < 4.5) \
    & (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
narrowed_cities_list_df = narrowed_cities_list_df.dropna()

# Display sample data
narrowed_cities_list_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_cities_list_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 15
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat= row['Lat']
    lon= row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon}, {lat}, {radius}'
    params["bias"] = f'proximity:{lon}, {lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    alpha = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)